In [1]:
import pandas as pd
import numpy as np


In [2]:
e1 = {"accuracy": 8, "altitude": 212.29999999999998, "heading": 0, 
"latitude": 32.11485166666667, 
"longitude": 34.81834666666667, "speed": 0}
e2 = {"accuracy": 8, "altitude": 212.2, "heading": 0, "latitude": 32.114844999999995, "longitude": 34.81836, "speed": 0}
e3 = {"accuracy": 8, "altitude": 211.29999999999998, "heading": 0, "latitude": 32.11484666666667, "longitude": 34.818371666666664, "speed": 0}
e4 = {"accuracy": 8.100000381469727, "altitude": 210.79999999999998, "heading": 0, "latitude": 
32.11484, "longitude": 
34.81838666666667, "speed": 0}
e5 = {"accuracy": 8, "altitude": 211.39999999999998, "heading": 0, "latitude": 32.114823333333334, "longitude": 34.818443333333335, "speed": 0}
e6 = {"accuracy": 8.100000381469727, "altitude": 211.5, "heading": 0, "latitude": 32.11481166666667, "longitude": 34.818441666666665, "speed": 0}

In [3]:
area = [
    {"latitude": 32.11344833333333, "longitude": 34.817755000000005}, 
    {"latitude": 32.11331833333334, "longitude": 34.81849833333333}, 
    {"latitude": 32.112855, "longitude": 34.817658333333334}, 
    {"latitude": 32.11272592560912, "longitude": 34.818381552078776}
]

In [90]:
topleft = {"accuracy": 1.399999976158142, "altitude": 31.4, "heading": 219.30999755859375, "latitude": 32.113566666666664, "longitude": 34.817744999999995, "speed": 0}
topright = {"accuracy": 1.5, "altitude": 29.4, "heading": 73.6500015258789, "latitude": 32.11342166666667, "longitude": 34.818509999999996, "speed": 0}
bottomleft = {"accuracy": 1.7000000476837158, "altitude": 27.5, "heading": 130.33999633789062, "latitude": 32.112835000000004, "longitude": 34.817706666666666, "speed": 0}
bottomright = {"accuracy": 1.7000000476837158, "altitude": 27.5, "heading": 130.33999633789062, "latitude": 32.11269000000001, "longitude": 34.81847166666667, "speed": 0}
area = [
    topleft,
    topright,
    bottomleft,
    bottomright
]
area = [{
    "latitude":a['latitude'],
    "longitude":a['longitude']
    } for a in area]
print(area)
area = [(a['latitude'],a['longitude']) for a in area]

area

[{'latitude': 32.113566666666664, 'longitude': 34.817744999999995}, {'latitude': 32.11342166666667, 'longitude': 34.818509999999996}, {'latitude': 32.112835000000004, 'longitude': 34.817706666666666}, {'latitude': 32.11269000000001, 'longitude': 34.81847166666667}]


[(32.113566666666664, 34.817744999999995),
 (32.11342166666667, 34.818509999999996),
 (32.112835000000004, 34.817706666666666),
 (32.11269000000001, 34.81847166666667)]

In [88]:

p = {"accuracy": 2, "altitude": 26.2, "heading": 32.77000045776367, "latitude": 32.11347833333333, "longitude": 34.81811166666667, "speed": 0}
p = (p['latitude'],p['longitude'])

In [89]:
def calculate_relative_position(new_point, area_coords):
    # Assuming new_point is in (latitude, longitude) format
    new_lat, new_lon = new_point
    
    # Top left coordinates
    tl_lat, tl_lon = area_coords[0]
    
    # Calculate relative position
    rel_lat = (new_lat - tl_lat) / (area_coords[2][0] - tl_lat)
    rel_lon = (new_lon - tl_lon) / (area_coords[1][1] - tl_lon)
    x = rel_lon * 100
    y = rel_lat * 100

    if y > 100:
        y = 100

    if y < 0:
        y = 0

    if x > 100:
        x = 100
    if x < 0:
        x = 0
        
    
    return {
        "relative_latitude_percent": rel_lat * 100,
        "relative_longitude_percent": rel_lon * 100,
        "x":x,
        "y":y
    }

calculate_relative_position(p,area)

{'relative_latitude_percent': 12.072892938235551,
 'relative_longitude_percent': 47.93028322498568,
 'x': 47.93028322498568,
 'y': 12.072892938235551}

Relative Position (Inside): {'relative_latitude_percent': 100, 'relative_longitude_percent': 33.33333333364294}
Relative Position (Outside in Latitude): {'relative_latitude_percent': 100, 'relative_longitude_percent': 33.33333333364294}
Relative Position (Outside in Longitude): {'relative_latitude_percent': 100, 'relative_longitude_percent': 100}


{'relative_latitude_percent': 12.072892938235551,
 'relative_longitude_percent': 47.93028322498568,
 'x': 47.93028322498568,
 'y': 12.072892938235551}

In [51]:
e1

{'accuracy': 8,
 'altitude': 212.29999999999998,
 'heading': 0,
 'latitude': 32.11485166666667,
 'longitude': 34.81834666666667,
 'speed': 0}

In [14]:
vec_top = np.array([topright["latitude"] - topleft["latitude"], topright["longitude"] - topleft["longitude"]])
vec_left = np.array([bottomleft["latitude"] - topleft["latitude"], bottomleft["longitude"] - topleft["longitude"]])


In [15]:
bottomright = {
    "latitude": topleft["latitude"] + vec_top[0] + vec_left[0],
    "longitude": topleft["longitude"] + vec_top[1] + vec_left[1]
}

In [16]:
bottomright['latitude'],bottomright['longitude']

(32.11269000000001, 34.81847166666667)

In [ ]:
area = [
    topleft,
    topright,
    bottomleft,
    bottomright
]

[(32.113566666666664, 34.817744999999995),
 (32.11342166666667, 34.818509999999996),
 (32.112835000000004, 34.817706666666666)]

In [ ]:
area2 = [
    {"latitude": 32.1135, "longitude": 34.817755000000005}, 
    {"latitude": 32.11331833333334, "longitude": 34.81849833333333}, 
    {"latitude": 32.112855, "longitude": 34.817658333333334}, 
    {"latitude": 32.11272592560912, "longitude": 34.818381552078776}
]

In [18]:
new_point = {
"latitude": 32.11485166666667, 
"longitude": 34.81834666666667,
}


In [17]:
def calculate_relative_position(topleft, bottomright, new_point):
    width = bottomright["longitude"] - topleft["longitude"]
    height = topleft["latitude"] - bottomright["latitude"]

    rel_lat = (new_point["latitude"] - topleft["latitude"]) / height
    rel_lon = (new_point["longitude"] - topleft["longitude"]) / width

    return {
        "relative_latitude_percent": rel_lat * 100,
        "relative_longitude_percent": rel_lon * 100
    }

In [19]:
calculate_relative_position(topleft,bottomright,new_point)

{'relative_latitude_percent': 146.57794677017492,
 'relative_longitude_percent': 82.79816513815517}

In [20]:
def find_percentage_coordinates(point, area):
    # Extract latitude and longitude from the point
    lat = point["latitude"]
    lon = point["longitude"]
    
    # Extract all latitude and longitude values from the area
    lats = [p["latitude"] for p in area]
    lons = [p["longitude"] for p in area]
    
    # Calculate min and max values for latitude and longitude
    min_lat = min(lats)
    max_lat = max(lats)
    min_lon = min(lons)
    max_lon = max(lons)
    
    # Calculate normalized coordinates within the bounds
    lat_percent = (lat - min_lat) / (max_lat - min_lat) * 100
    lon_percent = (lon - min_lon) / (max_lon - min_lon) * 100
    # Return the percentage coordinates as a tuple
    return lat_percent, lon_percent
    

# Calculate percentage coordinates for point e6 within the area
lat_percent, lon_percent = find_percentage_coordinates(e6, area)

print(f"Latitude %: {lat_percent:.4f}, Longitude %: {lon_percent:.4f}")


Latitude %: 270.1595, Longitude %: 91.4938


In [42]:
p = (e6['latitude'], e6['longitude'])
p

(32.11481166666667, 34.818441666666665)

In [29]:
# lat = y  - 0
# long = x - 1

In [46]:
i = 1
j = 0

In [ ]:
topleft[0] - topright[0]

In [47]:
(topleft[i] - p[i]) / (topleft[i] - topright[i]), (topleft[j] - p[j]) / (topleft[j] - bottomleft[j])


(0.9106753812663686, -1.7015945330518227)

((32.113566666666664, 34.817744999999995),
 (32.11342166666667, 34.818509999999996))

In [91]:
csv_file = 'gps_data.csv'
data = pd.read_csv(csv_file)

In [93]:
data['heading'].max()

359.82000732421875

In [22]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 3.4 MB/s eta 0:00:00m eta 0:00:010:00:01


In [23]:
import numpy as np
import cv2

# Given points
topleft = (32.113566666666664, 34.817744999999995)
topright = (32.11342166666667, 34.818509999999996)
bottomleft = (32.112835000000004, 34.817706666666666)

# Calculate the fourth point (bottom right) using vector addition
vec_top = np.array([topright[0] - topleft[0], topright[1] - topleft[1]])
vec_left = np.array([bottomleft[0] - topleft[0], bottomleft[1] - topleft[1]])
bottomright = (topleft[0] + vec_left[0] + vec_top[0], topleft[1] + vec_left[1] + vec_top[1])

# Original points (source points)
src_points = np.array([topleft, topright, bottomleft, bottomright], dtype=np.float32)

# Destination points (points of a perfect rectangle)
dst_points = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
], dtype=np.float32)

# Compute the homography matrix
h_matrix, _ = cv2.findHomography(src_points, dst_points)

# Function to calculate the relative position of a new point
def calculate_relative_position(new_point, h_matrix):
    point = np.array([[new_point[0], new_point[1]]], dtype=np.float32)
    point = np.array([point])
    transformed_point = cv2.perspectiveTransform(point, h_matrix)
    rel_lat = transformed_point[0][0][1]
    rel_lon = transformed_point[0][0][0]
    return {
        "relative_latitude_percent": rel_lat * 100,
        "relative_longitude_percent": rel_lon * 100
    }

# New point (example)
new_point = (32.113, 34.818)

# Calculate the relative position
relative_position = calculate_relative_position(new_point, h_matrix)

print("Bottom Right Point:", bottomright)
print("Relative Position:", relative_position)


Bottom Right Point: (32.11269000000001, 34.81847166666667)
Relative Position: {'relative_latitude_percent': 70.27849555015564, 'relative_longitude_percent': 37.01392412185669}


In [7]:
data.columns

Index(['version', 'buildingId', 'x', 'y', 'floor', 'timestamp', 'accuracy',
       'longitude', 'latitude', 'altitude', 'heading', 'speed', 'meanCn0',
       'maxCn0', 'satellites', 'uploaddAt'],
      dtype='object')

In [19]:
data.groupby('floor').apply(lambda x: (x['altitude'] * x['accuracy']).sum() / x['accuracy'].sum())


floor
-1    43.997962
 0    38.811019
dtype: float64

In [18]:
data[['x','y','floor','altitude','accuracy']]

,x,y,floor,altitude,accuracy
0,1326,30,0,37.4,1.5
1,1326,30,0,37.4,1.5
2,1226,30,0,36.7,1.5
3,1106,30,0,41.2,1.6
4,1011,35,0,43.1,1.6
...,...,...,...,...,...
171,135,521,-1,44.8,2.3
172,35,381,-1,47.9,2.0
173,35,316,-1,50.1,1.8
174,175,316,-1,51.4,3.5


In [13]:
data.loc[data['floor','altitude']]

KeyError: ('floor', 'altitude')

In [6]:
# weighted_avg = data.groupby('floor').apply(lambda x: (x['altitude'] * x['altitude_accuracy']).sum() / x['altitude_accuracy'].sum())
# print(weighted_avg)

KeyError: 'altitude_accuracy'

In [5]:
lowest = data.loc[data['altitude'].idxmin()] 
highest = data.loc[data['altitude'].idxmax()]  
lowest['altitude'],highest['altitude']

(12.9, 58.900000000000006)

In [21]:
leftest_topest = data.loc[data['longitude'].idxmin()]  # Leftmost
rightest_topest = data.loc[data['longitude'].idxmax()]  # Rightmost
leftest_downest = data.loc[data['latitude'].idxmax()]  # Bottommost
print("Leftmost-Topmost Point: (x:{}, y:{}, floor:{}, longitude: {}, latitude: {})".format(leftest_topest['x'],leftest_topest['y'],leftest_topest['floor'],leftest_topest['longitude'], leftest_topest['latitude']))
print("Rightmost-Topmost Point: (x:{}, y:{}, floor:{}, longitude: {}, latitude: {})".format(rightest_topest['x'],rightest_topest['y'],rightest_topest['floor'],rightest_topest['longitude'], rightest_topest['latitude']))
print("Leftmost-Bottommost Point: (x:{}, y:{}, floor:{}, longitude: {}, latitude: {})".format(leftest_downest['x'],leftest_downest['y'],leftest_downest['floor'],leftest_downest['longitude'], leftest_downest['latitude']))


Leftmost-Topmost Point: (x:175, y:316, floor:-1, longitude: 34.81756666666667, latitude: 32.113205)
Rightmost-Topmost Point: (x:1326, y:30, floor:0, longitude: 34.81849833333333, latitude: 32.11331833333334)
Leftmost-Bottommost Point: (x:557, y:308, floor:0, longitude: 34.81800333333333, latitude: 32.113953333333335)


In [26]:
leftest_topest = data.loc[data['x'].idxmin()] # Leftmost
rightest_topest = data.loc[data['x'].idxmax()] # Rightmost
leftest_downest = data.loc[data['y'].idxmax()] # Bottommost
leftest_topest[['x','y','longitude','latitude']],rightest_topest[['x','y','longitude','latitude']],leftest_downest[['x','y','longitude','latitude']]

(x                    0
 y                   35
 longitude    34.817755
 latitude     32.113448
 Name: 24, dtype: object,
 x                 1326
 y                   30
 longitude    34.818498
 latitude     32.113318
 Name: 1, dtype: object,
 x                   35
 y                 1181
 longitude    34.817658
 latitude     32.112855
 Name: 167, dtype: object)

In [39]:
lt = leftest_topest[['x','y','longitude','latitude']].to_dict()
rt  = rightest_topest[['x','y','longitude','latitude']].to_dict()
lb = leftest_downest[['x','y','longitude','latitude']].to_dict()
print(lt,rt,lb)

{'x': 0, 'y': 35, 'longitude': 34.817755000000005, 'latitude': 32.11344833333333} {'x': 1326, 'y': 30, 'longitude': 34.81849833333333, 'latitude': 32.11331833333334} {'x': 35, 'y': 1181, 'longitude': 34.817658333333334, 'latitude': 32.112855}


In [40]:


# Define the known points
# lt = {'x': 0, 'y': 35, 'longitude': 34.817755, 'latitude': 32.113448}
# rt = {'x': 1326, 'y': 30, 'longitude': 34.818498, 'latitude': 32.113318}
# lb = {'x': 35, 'y': 1181, 'longitude': 34.817658, 'latitude': 32.112855}

# Coordinates of the bounding points
points = np.array([
    [lt['x'], lt['y']],
    [rt['x'], rt['y']],
    [lb['x'], lb['y']]
])

# Longitudes and latitudes of the bounding points
longitudes = np.array([lt['longitude'], rt['longitude'], lb['longitude']])
latitudes = np.array([lt['latitude'], rt['latitude'], lb['latitude']])

# Define the point to be transformed
point_to_transform = np.array([1326, 1181])

# Calculate the transformation using linear algebra
# Add a column of ones for the affine transformation
points_ones = np.hstack([points, np.ones((points.shape[0], 1))])
transform_matrix_longitude = np.linalg.lstsq(points_ones, longitudes, rcond=None)[0]
transform_matrix_latitude = np.linalg.lstsq(points_ones, latitudes, rcond=None)[0]

# Add a 1 to the point to be transformed
point_to_transform_ones = np.hstack([point_to_transform, 1])

# Calculate the transformed coordinates
transformed_longitude = np.dot(point_to_transform_ones, transform_matrix_longitude)
transformed_latitude = np.dot(point_to_transform_ones, transform_matrix_latitude)

print("Rightmost-Bottommost Point: (longitude: {}, latitude: {})".format(transformed_longitude, transformed_latitude))


Rightmost-Bottommost Point: (longitude: 34.818381552078776, latitude: 32.11272592560912)


In [19]:
# Coordinates of the bounding points
leftest_topest_coord = np.array([leftest_topest['x'], leftest_topest['y']])
rightest_topest_coord = np.array([rightest_topest['x'], rightest_topest['y']])
leftest_downest_coord = np.array([leftest_downest['x'], leftest_downest['y']])

# Calculate the vector from leftest_topest to rightest_topest
vector_top = rightest_topest_coord - leftest_topest_coord

# Assuming the rectangular map, calculate the vector from leftest_topest to leftest_downest
vector_left = leftest_downest_coord - leftest_topest_coord

# Calculate the rightest-downest point
rightest_downest_coord = leftest_downest_coord + vector_top

print("Rightmost-Bottommost Point: (x: {}, y: {})".format(rightest_downest_coord[0], rightest_downest_coord[1]))

Rightmost-Bottommost Point: (x: 1361, y: 1176)
